# Functional Test 3.1.2 - GPUs

This Jupyter notebook will allow you to create VMs on different sites and worker nodes consistent with requirements for test 3.1.2 for testing GPU attachment.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library

In [2]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

-----------------------------------  --------------------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /home/fabric/.tokens.json
project_id                           990d8a8b-7e50-4d13-a3be-0f133ffa8653
bastion_username                     ibaldin_0000241998
bastion_key_filename                 /home/fabric/work/fabric_config/fabric_bastion_key
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /home/fabric/work/fabric_config/slice_key.pub
slice_private_key_file               /home/fabric/work/fabric_config/slice_key
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  --------------------------------------------------


## Step 3 Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [3]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the test Slice

This creates a VM with a GPU attached on a specific worker at a specific site. Depending on which worker you are using different types or no GPUs may be available. If you are unsure, the generated ads for each site ([in JSON format](https://github.com/fabric-testbed/aggregate-ads/tree/main/JSON)) can help. 

**You should try different nodes and different GPU types**

**The code to create the slice will auto-refresh until the slice is created or it fails**

In [16]:
from datetime import datetime
from dateutil import tz

name='Node1'
gpu_name='gpu1'
site='TACC'
# since all workers have a standard naming scheme, you can just change the worker
# to move from worker to worker
worker=f'{site.lower()}-w1.fabric-testbed.net'
cores=10
ram=20
disk=50
slice_name=f"Slice Test 3.1.2-GPU {site} on {worker} on {datetime.now()}"

In [7]:
try:
    #Create Slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=name, site=site, host=worker, cores=cores, ram=ram, disk=disk)
    
    # Add a GPU by type - be sure to select the righ kind for that worker
    #node.add_component(model='GPU_TeslaT4', name=gpu_name)
    # OR
    node.add_component(model='GPU_RTX6000', name=gpu_name)

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


-----------  ---------------------------------------------------------------------------
Slice Name   Slice Test TACC on tacc-w1.fabric-testbed.net on 2022-08-31 01:09:04.792628
Slice ID     efec5ff9-0099-4f72-9009-a1ab1f55c44e
Slice State  StableOK
Lease End    2022-09-01 01:09:13 +0000
-----------  ---------------------------------------------------------------------------

Retry: 13, Time: 142 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image            Management IP    State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  ---------------  ---------------  -------  -------
c70530c8-4bf6-415c-8f4a-9a0dbf2f183c  Node1   TACC    tacc-w1.fabric-testbed.net       10     32     100  default_rocky_8  129.114.110.76   Active

Time to stable 142 seconds
Running post_boot_config ... Time to post boot config 143 seconds


## Step 5: Observe the Slice's Attributes

### Print the slice 

In [8]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ---------------------------------------------------------------------------
Slice Name   Slice Test TACC on tacc-w1.fabric-testbed.net on 2022-08-31 01:09:04.792628
Slice ID     efec5ff9-0099-4f72-9009-a1ab1f55c44e
Slice State  StableOK
Lease End    2022-09-01 01:09:13 +0000
-----------  ---------------------------------------------------------------------------


### Print the node

Each node in the slice has a set of get functions that return the node's attributes. Use the returned `SSH Command` string to check the node. You can do it from a Bash launched inside the Jupyter container.


In [10]:
try:
    node = slice.get_node(name) 
    print(f"{node}")
  
    gpu1 = node.get_component(gpu_name)
    print(f"{gpu1}")
    
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------------------------------------------------------------------------------------------
ID                 c70530c8-4bf6-415c-8f4a-9a0dbf2f183c
Name               Node1
Cores              10
RAM                32
Disk               100
Image              default_rocky_8
Image Type         qcow2
Host               tacc-w1.fabric-testbed.net
Site               TACC
Management IP      129.114.110.76
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -J ibaldin_0000241998@bastion-1.fabric-testbed.net rocky@129.114.110.76
-----------------  ------------------------------------------------------------------------------------------------------------------------
-----------  ----------------------------------------------------------
Name         Node1-gpu1
Details      NVIDIA Corporation TU102GL [Quadro RTX 6000/8000] (rev a1)
Disk (G)     0
Units        1
PCI Address  0000:25:00.0
Mode

### GPU PCI Device

Run the command <code>lspci</code> to see your GPU PCI device(s). This is the raw GPU PCI device that is not yet configured for use.  You can use the GPUs as you would any GPUs.

View node1's GPU

In [11]:
command = "sudo dnf install -q -y pciutils && lspci | grep 'NVIDIA\|3D controller'"
try:
    stdout, stderr = node.execute(command)
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

stdout: 
Installed:
  pciutils-3.7.0-1.el8.x86_64                                                   

00:07.0 3D controller: NVIDIA Corporation TU102GL [Quadro RTX 6000/8000] (rev a1)



## Step 6: Install Nvidia Drivers

Now, let's run the following commands to install the latest CUDA driver and the CUDA libraries and compiler.

In [12]:
commands = [
    'sudo dnf install -q -y epel-release',
    'sudo dnf config-manager --add-repo https://developer.download.nvidia.com/compute/cuda/repos/rhel8/x86_64/cuda-rhel8.repo',
    'sudo dnf install -q -y kernel-devel kernel-headers nvidia-driver nvidia-settings cuda-driver cuda'
]
try:
    print("Installing CUDA...")
    for command in commands:
        stdout, stderr = node.execute(command)
    print("Done installing CUDA. Now, reboot for the changes to take effect.")
except Exception as e:
    print(f"Fail: {e}")

Installing CUDA...
Done installing CUDA. Now, reboot for the changes to take effect.


And once CUDA is installed, reboot the machine.

In [13]:
reboot = 'sudo reboot'
try:
    print(reboot)
    node.execute(reboot)
    
    slice.wait_ssh(timeout=360,interval=10,progress=True)

    print("Now testing SSH abilites to reconnect...",end="")
    slice.update()
    slice.test_ssh()
    print("Reconnected!")

except Exception as e:
    print(f"Fail: {e}")

sudo reboot
Waiting for slice . Slice state: StableOK
Waiting for ssh in slice ... ssh successful
Now testing SSH abilites to reconnect...Reconnected!


## Step 7: Testing the GPU and CUDA Installation

Verify that the Nvidia drivers recognize the GPU by running `nvidia-smi`.

In [14]:
try:
    stdout, stderr = node.execute("nvidia-smi")
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

stdout: Wed Aug 31 01:23:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:00:07.0 Off |                    0 |
| N/A   30C    P0    54W / 250W |      0MiB / 23040MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               


## Step 8: Cleanup Your Slice

In [15]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")